In [1]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import random
from random import sample
import numpy as np
import time
from sklearn import svm
from sklearn import preprocessing
from sklearn import metrics
from sklearn import cross_validation
from sklearn import grid_search
import pywt

In [2]:
# SVM CLASSIFICATION FUNCTIONS
def randomSplit(X,y):
    
    accuracy = np.array([])
    # do 20 random splits
    for i in range(100):
        # leave 20% out for testing
        skf = cross_validation.StratifiedKFold(y,n_folds=10,shuffle=True) 
       
        for cv_i,test_i in skf:
            train_X = X[cv_i]
            train_y = y[cv_i]
            # do training here
            clf = gridSearchSVM(train_X,train_y)

            # do testing here
            test_X = X[test_i]
            test_y = y[test_i]
            prediction = clf.predict(test_X)
            print clf.best_params_
            # record performance
            foo = performance(prediction, test_y)
            print i
            print "Accuracy: " , foo
            accuracy = np.append(accuracy, foo)
            break
        
    mean_acc = np.mean(accuracy)
    return mean_acc


def gridSearchSVM(X,y):
    #parameters = {'kernel':('linear','rbf'), 'C':[1, 2, 3, 5, 10, 13, 15,20]}
    parameters = {'kernel':('linear','rbf'), 'C':[1, 2]}
    svr  = svm.SVC()
    clf = grid_search.GridSearchCV(svr, parameters, cv = 5, n_jobs = 1, verbose=1)
    clf.fit(X,y)
    return clf

def performance(prediction, target):
    acc = metrics.accuracy_score(target, prediction, normalize=True)
    return acc

In [3]:
# DATA LOADING FUNCTIONS
def loadYaleData():
    X = np.matrix(scipy.io.loadmat('yalefacesFFT.mat')['DATA'])
    individuals = 15; 
    picsPerInd = 11;
    Y = np.zeros((individuals*picsPerInd))
    for i in range(0,individuals):
        Y[(i)*picsPerInd:(i+1)*picsPerInd] = i;
    return (X,Y)

def loadOlivettiData():
    X = np.matrix(scipy.io.loadmat('olivettifacesFFT.mat')['DATA'])
    individuals = 40; 
    picsPerInd = 10;
    Y = np.zeros((individuals*picsPerInd))
    for i in range(0,individuals):
        Y[(i)*picsPerInd:(i+1)*picsPerInd] = i;
    return (X,Y)



In [4]:
# TRANFORMATION FUNCTIONS

def removePhase(V):
    return V[0:V.shape[0]/2].reshape(V.shape[0]/2);

def recFilter(w,h,fw,fh):
    h = h + 1
    F = np.zeros((h,w));
    for i in range((w-1)/2+1 - fw/2, (w-1)/2+1 + fw/2 - 1):
        for j in range(h - fh - 1, h):
            F[j,i] = 1
    return np.reshape(F,(w*h),order='F')

def triFilter(w,h,fw,fh):
    h = h + 1
    F = np.zeros((h,w));
    for j in range(h - fh -1, h):
        span = (j - (h - fh)) * (fw/2)/fh;
        for i in range((w-1)/2+1 - span, (w-1)/2+1 + span - 1):
            F[j,i] = 1
    return np.reshape(F,(w*h),order='F')

In [5]:
# RECONSTRUCTION FUNCTIONS

# for level 1 wavelet 
def getWLface(V):
    filterLen = len(V)/4
    sideLen = np.sqrt(filterLen)
    
    # set reshape to FORTRAN order since that is what matlab uses
    cA = np.reshape(V[0:filterLen], (sideLen,sideLen),order='F')
    cH = np.reshape(V[filterLen:2*filterLen], (sideLen,sideLen),order='F')
    cV = np.reshape(V[2*filterLen:3*filterLen], (sideLen,sideLen),order='F')
    cD = np.reshape(V[3*filterLen:], (sideLen,sideLen),order='F')
    
    plt.imshow(cH);
    plt.gray()
    plt.show()

    print cA.shape
    img = pywt.idwt2((cA, (cH, cV, cD)), 'db4');      
    
    return img


# Reconstruct an image from the Yale database 
# based on the all frequecies and phases
def getYaleface(V):
    R = V[0:63*32];
    theta = V[63*32:];

    # set reshape to FORTRAN order since that is what matlab uses
    R = np.reshape(R, (32,63),order='F')
    theta = np.reshape(theta, (32,63),order='F')

    FFTimg = np.multiply(R,np.exp(np.sqrt(-1+0j)*theta));
    FFTimg = np.concatenate((FFTimg, np.conj(np.rot90(FFTimg[0:31,:],2))), axis=0);       
    img = np.fft.ifft2(np.fft.ifftshift(FFTimg));
    
    return np.real(img)

def getYaleFFTImg(V):
    R = V[0:63*32];
    theta = V[63*32:];

    # set reshape to FORTRAN order since that is what matlab uses
    R = np.reshape(R, (32,63),order='F')
    theta = np.reshape(theta, (32,63),order='F')

    FFTimg = np.multiply(R,np.exp(np.sqrt(-1+0j)*theta));
    FFTimg = np.concatenate((FFTimg, np.conj(np.rot90(FFTimg[0:31,:],2))), axis=0); 
    return np.log(FFTimg + 1);

# Reconstruct an image from the Olivetti database 
# based on the all frequecies and phases
def getOlivettiface(V):
    R = V[0:63*32];
    theta = V[63*32:];

    # set reshape to FORTRAN order since that is what matlab uses
    R = np.reshape(R, (32,63),order='F')
    theta = np.reshape(theta, (32,63),order='F')

    FFTimg = np.multiply(R,np.exp(np.sqrt(-1+0j)*theta));
    FFTimg = np.concatenate((FFTimg, np.conj(np.rot90(FFTimg[0:31,:],2))), axis=0);       
    img = np.fft.ifft2(np.fft.ifftshift(FFTimg));
    
    return np.real(img)

def getOlivettiFFTImg(V):
    R = V[0:63*32];
    theta = V[63*32:];

    # set reshape to FORTRAN order since that is what matlab uses
    R = np.reshape(R, (32,63),order='F')
    theta = np.reshape(theta, (32,63),order='F')

    FFTimg = np.multiply(R,np.exp(np.sqrt(-1+0j)*theta));
    FFTimg = np.concatenate((FFTimg, np.conj(np.rot90(FFTimg[0:31,:],2))), axis=0); 
    return np.log(FFTimg + 1);

def addZeroPhase(V,):
    F = np.zeros((V.shape[0]*2))
    # add magnitude 
    F[0:V.shape[0]] = V[0:V.shape[0]].reshape(V.shape[0]);
    # add original phase
    #F[V.shape[0]:] = X[V.shape[0]:, i].reshape(V.shape[0]);
    # add zero phase
    F[V.shape[0]:] = 0;
    return F.reshape(F.shape[0]);

In [6]:
X = np.matrix(scipy.io.loadmat('yalefacesWL1.mat')['DATA_LV1'])
img = getWLface(X[:,1])
plt.imshow(img);
plt.gray()
plt.show()

(35L, 35L)


In [7]:
# DISPLAY FUNCTIONS
# h,w are height and width of the individual images, nh,nw are the dimensions of the 
# collage interms of images 
def makeCollage(h, w, nh, nw, DATA, randImg, getImage):
    
    border = 10
    collage = np.zeros((h*nh+border*(nh+1),w*nw+border*(nw+1)));
    #randImg = sample(xrange(DATA.shape[1]), nw*nh)

    for i in range(0,nw):
        for j in range(0,nh):
            img = getImage(DATA[:,randImg[nw*i + j]])
            collage[(i+1)*border-1+i*h:(i+1)*h+(i+1)*border-1, (j+1)*border+j*w-1:(j+1)*border-1+(j+1)*w] = img
    
    return collage
    

In [38]:
# BASELINE CLASSIFICATION
(X,Y) = loadYaleData()
print "Mean acc: " + str(randomSplit(np.transpose(X),Y))

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    2.2s finished


Fitting 5 folds for each of 4 candidates, totalling 20 fits
{'kernel': 'linear', 'C': 1}

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    2.2s finished



Accuracy:  0.8
Fitting 5 folds for each of 4 candidates, totalling 20 fits
{'kernel': 'linear', 'C': 1}

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    2.2s finished


...


Accuracy:  0.8
Fitting 5 folds for each of 4 candidates, totalling 20 fits
{'kernel': 'linear', 'C': 1}

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    2.3s finished



Accuracy:  0.8
Fitting 5 folds for each of 4 candidates, totalling 20 fits
{'kernel': 'linear', 'C': 1}

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    2.3s finished



Accuracy:  0.666666666667
Fitting 5 folds for each of 4 candidates, totalling 20 fits
{'kernel': 'linear', 'C': 1}
Accuracy:  0.833333333333
Mean acc: 0.741666666667


In [25]:
(X,Y) = loadYaleData() 
plt.imshow(makeCollage(63, 63, 10, 10, X, random.sample(range(Y.shape[0]), 100), getYaleface));
plt.gray()
plt.show()

(X,Y) = loadOlivettiData()
plt.imshow(makeCollage(63, 63, 10, 10, X, random.sample(range(Y.shape[0]), 100), getYaleface));
plt.gray()
plt.show()

In [40]:
# NO PHASE CLASSIFICATION
yaleHeight = 63
yaleWidth = 63
olivettHeight = 63
olivettWidth = 63

(X,Y) = loadYaleData()


xNoPhase = np.zeros([X.shape[0]/2, X.shape[1]])

for i in range(0,X.shape[1]):
    xNoPhase[:,i] = removePhase(X[:,i])[:]

xZeroPhase = np.zeros([X.shape[0], X.shape[1]])
for i in range(0,X.shape[1]):
    xZeroPhase[:,i] = addZeroPhase(xNoPhase[:,i])[:]

print X.shape
print xNoPhase.shape
print xZeroPhase.shape

randImgs = sample(xrange(X.shape[1]), 4)
makeCollage(olivettHeight, olivettWidth, 1, 1, X, randImgs, getOlivettiface)
makeCollage(olivettHeight, olivettWidth, 1, 1, xZeroPhase, randImgs, getOlivettiface)

print "Mean acc: " + str(randomSplit(np.transpose(xNoPhase),Y))

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.1s finished


(4032L, 165L)
(2016L, 165L)
(4032L, 165L)
Fitting 5 folds for each of 4 candidates, totalling 20 fits
{'kernel': 'linear', 'C': 1}

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.1s finished



0
Accuracy:  0.766666666667
Fitting 5 folds for each of 4 candidates, totalling 20 fits
{'kernel': 'linear', 'C': 1}

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.0s finished


...


96
Accuracy:  0.766666666667
Fitting 5 folds for each of 4 candidates, totalling 20 fits
{'kernel': 'linear', 'C': 1}

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.1s finished



97
Accuracy:  0.766666666667
Fitting 5 folds for each of 4 candidates, totalling 20 fits
{'kernel': 'linear', 'C': 1}

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.0s finished



98
Accuracy:  0.866666666667
Fitting 5 folds for each of 4 candidates, totalling 20 fits
{'kernel': 'linear', 'C': 1}
99
Accuracy:  0.666666666667
Mean acc: 0.735


In [32]:
# Make Filtered Datasets
height = 63
width = 63

for w in range(5, 40, 5):
    (X,Y) = loadOlivettiData()
    F = recFilter(width, (height-1)/2, w, w)
    randImgs = [0, 72, 84, 9, 111]

    for i in range(0, X.shape[1]):
        for j in range(0, F.shape[0]):
            X[j,i] = X[j,i] * F[j]

    #print np.count_nonzero(X[0:F.shape[0]+1,0])
    np.save("olivettiF_R_" + str(w) + "_" + str(w), X)
    c = makeCollage(height, width, 1, 1, X, randImgs, getOlivettiFFTImg)
    scipy.misc.imsave('oFFT_R_' + str(w) + "_" + str(w) + '.jpg', c)
    c = makeCollage(height, width, 1, 1, X, randImgs, getOlivettiface)
    scipy.misc.imsave('oRecon_R_' + str(w) + "_" + str(w) + '.jpg', c)

C:\Users\AI\Anaconda\lib\site-packages\IPython\kernel\__main__.py:13: ComplexWarning: Casting complex values to real discards the imaginary part


In [112]:
# Run Classification on Filtered Data (Rectangle)
accuracy_phase = np.array([])
dim_phase = np.array([]) 
accuracy_nophase = np.array([])
dim_nophase = np.array([]) 

# load the labels 
(X,Y) = loadYaleData()

for w in range(10, 130, 10):
    # load filtered data 
    X = np.load("./Filters/FilteredData/yaleF_R_" + str(w) + "_" + str(w) + ".npy")
        
    X_nophase = np.zeros([X.shape[0]/2, X.shape[1]])
    for i in range(0,X.shape[1]):
        X_nophase[:,i] = removePhase(X[:,i])[:]
    
    
    accuracy_phase = np.append(accuracy_phase, randomSplit(np.transpose(X),Y))
    print accuracy_phase[-1]
    
    dim_phase = np.append(dim_phase, w*w + X.shape[0]/2) 
    print dim_phase[-1]
    
    accuracy_nophase = np.append(accuracy_nophase, randomSplit(np.transpose(X_nophase),Y))
    print accuracy_nophase[-1]
    
    dim_nophase = np.append(dim_nophase, w*w)
    print dim_nophase[-1]

np.savetxt("Y_R_acc_phase", accuracy_phase)
np.savetxt("Y_R_dim_phase", dim_phase)
np.savetxt("Y_R_acc_nophase", accuracy_nophase)
np.savetxt("Y_R_dim_phase", dim_nophase)


In [ ]:
# Run Classification on Filtered Data (Triangle)
accuracy_phase = np.array([])
dim_phase = np.array([]) 
accuracy_nophase = np.array([])
dim_nophase = np.array([]) 

# load the labels 
(X,Y) = loadOlivettiData()

for w in range(5, 40, 5):
    # load filtered data 
    X = np.load("./Filters/FilteredData/olivettiF_T_" + str(w) + "_" + str(w) + ".npy")
    
    #c = makeCollage(olivettHeight, olivettWidth, 1, 1, X, randImgs, getOlivettiFFTImg)
    #plt.imshow(c)
    #plt.show()
    
    X_nophase = np.zeros([X.shape[0]/2, X.shape[1]])
    for i in range(0,X.shape[1]):
        X_nophase[:,i] = removePhase(X[:,i])[:]
    
    
    accuracy_phase = np.append(accuracy_phase, randomSplit(np.transpose(X),Y))
    print accuracy_phase[-1]
    
    dim_phase = np.append(dim_phase, np.count_nonzero(X[0:X.shape[0]/2,0]) + X.shape[0]/2)
    print dim_phase[-1]
    
    accuracy_nophase = np.append(accuracy_nophase, randomSplit(np.transpose(X_nophase),Y))
    print accuracy_nophase[-1]
    
    dim_nophase = np.append(dim_nophase, np.count_nonzero(X[0:X.shape[0]/2,0]))
    print dim_nophase[-1]

np.savetxt("O_T_acc_phase", accuracy_phase)
np.savetxt("O_T_dim_phase", dim_phase)
np.savetxt("O_T_acc_nophase", accuracy_nophase)
np.savetxt("O_T_dim_nophase", dim_nophase)


In [ ]:
# Run Classification Testing

numExperiments = 14

accuracy_phase = np.zeros(numExperiments)
dim_phase = np.zeros(numExperiments)
accuracy_nophase = np.zeros(numExperiments)
dim_nophase = np.zeros(numExperiments) 

accuracy = np.array([])
numTests = 5

height = 63
width = 63
     
# load the labels 
(X,Y) = loadOlivettiData()

for i in range(numTests):
    # leave 10% out for testing
    skf = cross_validation.StratifiedKFold(Y,n_folds=10,shuffle=True) 

    for cv_i,test_i in skf:
        exp = 0
        
        for w in range(5, 40, 5):
            # load filtered data 
            X = np.load("./Filters/FilteredData/olivettiF_R_" + str(w) + "_" + str(w) + ".npy")

            X_nophase = np.zeros([X.shape[0]/2, X.shape[1]])
            for i in range(0,X.shape[1]):
                X_nophase[:,i] = removePhase(X[:,i])[:]
            
            # feature engineering and selection 
            F = recFilter(width, (height-1)/2, w, w)
            X_nophase = X_nophase[F == 1]
            X = X[np.append(F, np.ones(X.shape[0]/2), axis=0) == 1]
            
            test_y = Y[test_i]
            train_y = Y[cv_i]

            # full dim test 
            train_X = np.transpose(X)[cv_i]           
            test_X = np.transpose(X)[test_i]
            accuracy = trainTestSVM(train_X, train_y, test_X, test_y)
            accuracy_phase[exp] = accuracy_phase[exp] + accuracy
            dim_phase[exp] = sum(F) + X.shape[0]/2
            
            # no phase test 
            train_X = np.transpose(X_nophase)[cv_i]  
            test_X = np.transpose(X_nophase)[test_i]
            accuracy = trainTestSVM(train_X, train_y, test_X, test_y)
            accuracy_nophase[exp] = accuracy_nophase[exp] + accuracy
            dim_nophase[exp] = sum(F)
            
            exp = exp + 1
        
        for w in range(5, 40, 5):
            # load filtered data 
            X = np.load("./Filters/FilteredData/olivettiF_T_" + str(w) + "_" + str(w) + ".npy")

            X_nophase = np.zeros([X.shape[0]/2, X.shape[1]])
            for i in range(0,X.shape[1]):
                X_nophase[:,i] = removePhase(X[:,i])[:]
            
            # feature engineering and selection 
            F = triFilter(width, (height-1)/2, w, w)
            X_nophase = X_nophase[F == 1]
            X = X[np.append(F, np.ones(X.shape[0]/2), axis=0) == 1]
            
            test_y = Y[test_i]
            train_y = Y[cv_i]

            # full dim test 
            train_X = np.transpose(X)[cv_i]           
            test_X = np.transpose(X)[test_i]
            accuracy = trainTestSVM(train_X, train_y, test_X, test_y)
            accuracy_phase[exp] = accuracy_phase[exp] + accuracy
            dim_phase[exp] = sum(F) + X.shape[0]/2
            
            # no phase test 
            train_X = np.transpose(X_nophase)[cv_i]  
            test_X = np.transpose(X_nophase)[test_i]
            accuracy = trainTestSVM(train_X, train_y, test_X, test_y)
            accuracy_nophase[exp] = accuracy_nophase[exp] + accuracy
            dim_nophase[exp] = sum(F)
            
            exp = exp + 1
                     
        break
    
    print "Iterations " + str(i)

print accuracy_phase/numTests
print accuracy_nophase/numTests
print dim_phase
print dim_nophase

In [45]:
def varFilter(train_X, numFeatures):
    F = np.zeros(train_X.shape[1])
    
    var = np.var(train_X, axis=0)
    varSorted = np.sort(var)[::-1]
    
    F[var >= varSorted[numFeatures]] = 1
    
    img = np.reshape(F, (32, 63),order='F')
    plt.imshow(img)
    plt.gray()
    plt.show()
    
    return 0

(X,Y) = loadYaleData()
X_nophase = np.zeros([X.shape[0]/2, X.shape[1]])
for i in range(0,X.shape[1]):
    X_nophase[:,i] = removePhase(X[:,i])[:]

train_X = np.transpose(X_nophase);
varFilter(train_X, 200);           


In [56]:
def snrFilter(train_X, train_Y, numFeatures):
    F = np.zeros(train_X.shape[1])
    SNRsum = np.zeros(train_X.shape[1])
    numClasses = np.unique(train_Y).shape[0]
    
    for j in range(numClasses):
        pos = train_X[train_Y == j,:]
        neg = train_X[train_Y != j,:]
        mu_pos = np.mean(pos, axis = 0)
        mu_neg = np.mean(neg, axis = 0)
        sd_pos = np.std(pos, axis = 0)
        sd_neg = np.std(neg, axis = 0)
        SNRsum = SNRsum + np.abs((mu_pos - mu_neg)/(sd_pos + sd_neg))
        
    SNRavg = SNRsum/numClasses
    SNRavgSorted = np.sort(SNRavg)[::-1]
    
    F[SNRavg >= SNRavgSorted[numFeatures]] = 1
    
    img = np.reshape(F, (32, 63),order='F')
    plt.imshow(img)
    plt.gray()
    plt.show()
    
    return 0

(X,Y) = loadYaleData()
X_nophase = np.zeros([X.shape[0]/2, X.shape[1]])
for i in range(0,X.shape[1]):
    X_nophase[:,i] = removePhase(X[:,i])[:]

train_X = np.transpose(X_nophase);
snrFilter(train_X, Y, 200); 

In [58]:
def fdrFilter(train_X, train_Y, numFeatures):
    F = np.zeros(train_X.shape[1])
    FDRsum = np.zeros(train_X.shape[1])
    numClasses = np.unique(train_Y).shape[0]
    
    for j in range(numClasses):
        pos = train_X[train_Y == j,:]
        neg = train_X[train_Y != j,:]
        mu_pos = np.mean(pos, axis = 0)
        mu_neg = np.mean(neg, axis = 0)
        sd_pos = np.std(pos, axis = 0)
        sd_neg = np.std(neg, axis = 0)
        FDRsum = FDRsum + np.square(mu_pos - mu_neg)/(np.square(sd_pos) + np.square(sd_neg))
        
    FDRavg = FDRsum/numClasses
    FDRavgSorted = np.sort(FDRavg)[::-1]
    
    F[FDRavg >= FDRavgSorted[numFeatures]] = 1
    
    img = np.reshape(F, (32, 63),order='F')
    plt.imshow(img)
    plt.gray()
    plt.show()
    
    return 0

(X,Y) = loadYaleData()
X_nophase = np.zeros([X.shape[0]/2, X.shape[1]])
for i in range(0,X.shape[1]):
    X_nophase[:,i] = removePhase(X[:,i])[:]

train_X = np.transpose(X_nophase);
fdrFilter(train_X, Y, 200); 

In [69]:
def sdFilter(train_X, train_Y, numFeatures):
    F = np.zeros(train_X.shape[1])
    SDsum = np.zeros(train_X.shape[1])
    numClasses = np.unique(train_Y).shape[0]
    
    for j in range(numClasses):
        pos = train_X[train_Y == j,:]
        neg = train_X[train_Y != j,:]
        mu_pos = np.mean(pos, axis = 0)
        mu_neg = np.mean(neg, axis = 0)
        var_pos = np.var(pos, axis = 0)
        var_neg = np.var(neg, axis = 0)
        SDsum = SDsum + 0.5 * (var_pos/var_neg + var_neg/var_pos) + 0.5 * (np.square(mu_pos - mu_neg)/(var_pos + var_neg)) - 1
        
    SDavg = SDsum/numClasses
    SDavgSorted = np.sort(SDavg)[::-1]
    
    F[SDavg >= SDavgSorted[numFeatures]] = 1
    
    img = np.reshape(F, (32, 63),order='F')
    plt.imshow(img)
    plt.gray()
    plt.show()
    
    return 0

(X,Y) = loadYaleData()
X_nophase = np.zeros([X.shape[0]/2, X.shape[1]])
for i in range(0,X.shape[1]):
    X_nophase[:,i] = removePhase(X[:,i])[:]

train_X = np.transpose(X_nophase);
sdFilter(train_X, Y, 200);

In [ ]:
def tFilter(train_X, train_Y, numFeatures):
    F = np.zeros(train_X.shape[1])
    SDsum = np.zeros(train_X.shape[1])
    numClasses = np.unique(train_Y).shape[0]
    
    for j in range(numClasses):
        pos = train_X[train_Y == j,:]
        neg = train_X[train_Y != j,:]
        N_pos = pos.shape[0]
        N_neg = neg.shape[0]
        print pos.shape
        mu_pos = np.mean(pos, axis = 0)
        mu_neg = np.mean(neg, axis = 0)
        var_pos = np.var(pos, axis = 0)
        var_neg = np.var(neg, axis = 0)
        Tsum = Tsum + 0.5 * (var_pos/var_neg + var_neg/var_pos) + 0.5 * (np.square(mu_pos - mu_neg)/(var_pos + var_neg)) - 1
        
    Tavg = Tsum/numClasses
    TavgSorted = np.sort(Tavg)[::-1]
    
    F[Tavg >= TavgSorted[numFeatures]] = 1
    
    img = np.reshape(F, (32, 63),order='F')
    plt.imshow(img)
    plt.gray()
    plt.show()
    
    return 0

(X,Y) = loadYaleData()
X_nophase = np.zeros([X.shape[0]/2, X.shape[1]])
for i in range(0,X.shape[1]):
    X_nophase[:,i] = removePhase(X[:,i])[:]

train_X = np.transpose(X_nophase);
tFilter(train_X, Y, 200);

In [42]:
def trainTestSVM(train_X, train_y, test_X, test_y):

    clf = gridSearchSVM(train_X,train_y)

    prediction = clf.predict(test_X)
    print clf.best_params_
    # record performance
    accuracy = performance(prediction, test_y)
    print "Accuracy: " , accuracy
    
    return accuracy

In [ ]:
accuracy_phase = np.loadtxt("Y_R_acc_phase")
dim_phase = np.loadtxt("Y_R_dim_phase")
accuracy_nophase = np.loadtxt("Y_R_acc_nophase")
dim_nophase = np.loadtxt("Y_R_dim_nophase")

plt.title("Yale Rectanular Filter With Phase")
plt.xlabel("Dimension")
plt.ylabel("Mean Accuracy")
plt.scatter(dim_phase, accuracy_phase)
plt.show()

plt.title("Yale Rectanular Filter Without Phase")
plt.xlabel("Dimension")
plt.ylabel("Mean Accuracy")
plt.scatter(dim_nophase, accuracy_nophase)
plt.show()